In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

Ile kategorii filmów mamy w wypożyczalni?

In [2]:
df = pd.read_sql('select count(category_id) from category',con=connection)
df

,count
0,16


Wyświetl listę kategorii w kolejności alfabetycznej.

In [3]:
df = pd.read_sql('select name from category order by name ASC',con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [4]:
df = pd.read_sql('(select * from film order by release_year DESC limit 1) UNION ALL (select * from film order by release_year ASC limit 1)',con=connection)
df

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006,1,7,4.99,117,14.99,NC-17,2013-05-26 14:50:58.951,[Trailers],'chamber':1 'fate':4 'husband':11 'italian':2 ...
1,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006,1,7,4.99,117,14.99,NC-17,2013-05-26 14:50:58.951,[Trailers],'chamber':1 'fate':4 'husband':11 'italian':2 ...


Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [5]:
df = pd.read_sql("select count(rental_date) from rental WHERE rental_date BETWEEN '2005-07-01' AND '2005-08-01' ",con=connection)
df

,count
0,6709


Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [6]:
df = pd.read_sql("select count(rental_date) from rental WHERE rental_date BETWEEN '2010-01-01' AND '2011-02-01' ",con=connection)
df

,count
0,0


Znajdź największą płatność wypożyczenia.

In [7]:
df = pd.read_sql("select amount from payment ORDER BY amount DESC LIMIT 1",con=connection)
df

,amount
0,11.99


Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [8]:
df = pd.read_sql("select count(country) from customer left join address on customer.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id where country.country in('Poland','Nigeria','Bangladesh') ", con=connection)
df

,count
0,24


Gdzie mieszkają członkowie personelu?

In [9]:
df = pd.read_sql("select first_name, last_name, address from staff left join address on staff.address_id = address.address_id", con=connection)
df

,first_name,last_name,address
0,Mike,Hillyer,23 Workhaven Lane
1,Jon,Stephens,1411 Lillydale Drive


Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [10]:
df = pd.read_sql("select count(*) from staff left join address on staff.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id where country.country in('Argentina', 'Spain')", con=connection )
df

,count
0,0


Jakie kategorie filmów zostały wypożyczone przez klientów?

In [11]:
df = pd.read_sql("select distinct name from rental left join inventory on rental.inventory_id = inventory.inventory_id left join film_category on inventory.film_id = film_category.film_id left join category on film_category.category_id = category.category_id ", con=connection)
df

,name
0,Horror
1,Comedy
2,Drama
3,Sci-Fi
4,Foreign
5,Classics
6,Games
7,Travel
8,New
9,Music


Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [12]:
df2 = pd.read_sql("select distinct name from rental left join inventory on rental.inventory_id = inventory.inventory_id left join film_category on inventory.film_id = film_category.film_id left join category on film_category.category_id = category.category_id left join customer on rental.customer_id = customer.customer_id left join address on customer.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id where country like 'United States'", con=connection)
df2

,name
0,Horror
1,Sci-Fi
2,Drama
3,Comedy
4,Foreign
5,Classics
6,Games
7,New
8,Travel
9,Music


Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [20]:
df = pd.read_sql("select * from actor where (first_name = 'Olympia' and last_name = 'Pfeiffer') or (first_name = 'Julia' and last_name = 'Zellweger') or (first_name = 'Ellen' and last_name = 'Presley')",con=connection)
df

,actor_id,first_name,last_name,last_update
0,93,Ellen,Presley,2013-05-26 14:47:57.620
1,171,Olympia,Pfeiffer,2013-05-26 14:47:57.620
2,186,Julia,Zellweger,2013-05-26 14:47:57.620


In [22]:
df = pd.read_sql("select title, first_name, last_name from film inner join film_actor on film.film_id = film_actor.film_id  inner join actor on film_actor.actor_id = actor.actor_id where (first_name = 'Olympia' and last_name = 'Pfeiffer') or (first_name = 'Julia' and last_name = 'Zellweger') or (first_name = 'Ellen' and last_name = 'Presley')",con=connection)
df

,title,first_name,last_name
0,Bilko Anonymous,Ellen,Presley
1,Caribbean Liberty,Ellen,Presley
2,Casper Dragonfly,Ellen,Presley
3,Empire Malkovich,Ellen,Presley
4,Floats Garden,Ellen,Presley
...,...,...,...
64,Outlaw Hanky,Julia,Zellweger
65,Panky Submarine,Julia,Zellweger
66,Rider Caddyshack,Julia,Zellweger
67,Won Dares,Julia,Zellweger
